In [ ]:
# make sure mongodb is running

In [ ]:
import os
from scipy.signal import argrelextrema
import numpy as np
from pymongo import MongoClient
import cv2
import re
import utils.const as const

In [ ]:
poolSize=8

### most important
frameToExtract=10

###### make success and failure list
failed=[]
extracted=[]
###### make queue for each process

db=const.grpId
collection=const.PostCollection
directory=db

In [ ]:
client = MongoClient()
client.list_database_names() # ['admin', 'local']

In [ ]:
client = MongoClient()
myCollect=client[db][collection]

In [ ]:
## read logs for already extracted keyframes and create dictionary

alreadyExtracted={}

regex = re.compile('(extracted*.*txt$)')

for file in os.listdir():
  if regex.match(file):
    with open(file,'r') as fp:
      for line in fp.readlines():
        print(line[:-1])
        alreadyExtracted[line[:-1]]=1

In [ ]:
vids=[]
for post in myCollect.find({}):
  if '_id' in post:
    A=os.path.join(directory,post['_id'])

    if 'attachments' in post:
      B=os.path.join(A,'attachments')

      if 'videos' in post['attachments']:
        C=os.path.join(B,'videos')
        for i in range(len(post['attachments']['videos'])):
          D=os.path.join(C,str(i))
          if D not in alreadyExtracted:
            vids.append(D)
      
          
    
    if 'comments' in post:
      B=os.path.join(A,'comments')
      for comment in post['comments']:
        if '_id' in comment:
          C=os.path.join(B,comment['_id'])

          if '_id' and 'attachment' in comment:
            D=os.path.join(C,'attachment')

            if 'videos' in comment['attachment']:
              E=os.path.join(D,'videos')

              for i in range(len(comment['attachment']['videos'])):
                F=os.path.join(E,str(i))
                if F not in alreadyExtracted:
                  vids.append(F)


print(len(vids))
print(len(alreadyExtracted))

In [ ]:
order=[]

if os.path.isfile('order.txt'):
    print('order.txt exist')
    with open('order.txt','r') as fp:
        for line in fp.readlines():
            if len(line)<=1:
                continue
            order.append(line[:-1])
else:
    print('creating order.txt')
    dictOrder={}
    totalSize=0
    maxSize=0
    for i in vids:
        dictOrder[i]=os.stat(i).st_size
        totalSize+=dictOrder[i]
        maxSize=max(maxSize,dictOrder[i])
    
    print('Max size',maxSize)
    print('Total size',totalSize)
    
    dict2=dict(sorted(dictOrder.items(), key=lambda item: item[1]))

    with open('order.txt','w') as fp:
        for k,v in dict2.items():
            order.append(k)
            fp.write(k+'\n')

In [ ]:
vids=[]
for i in order:
    if i not in alreadyExtracted:
        vids.append(i)
print(len(vids))

In [ ]:
def smooth(x, window_len=13, window='hanning'):
    s = np.r_[2 * x[0] - x[window_len:1:-1],x, 2 * x[-1] - x[-1:-window_len:-1]]
    #print(len(s))

    if window == 'flat':  # moving average
        w = np.ones(window_len, 'd')
    else:
        w = getattr(np, window)(window_len)
    y = np.convolve(w / w.sum(), s, mode='same')
    return y[window_len - 1:-window_len + 1]

In [ ]:
def getDiff(name):
  cap = cv2.VideoCapture(name)

  curr_frame = None
  prev_frame = None

  frame_diffs = []
  ret, frame = cap.read()
  i = 1

  while(ret):
    luv = cv2.cvtColor(frame, cv2.COLOR_BGR2LUV)
    curr_frame = luv
    if curr_frame is not None and prev_frame is not None:
    #logic here
      diff = cv2.absdiff(curr_frame, prev_frame)
      count = np.sum(diff)
      frame_diffs.append(count)

    prev_frame = curr_frame
    i = i + 1
    ret, frame = cap.read()

  cap.release()

  return np.array(frame_diffs)

In [ ]:
def getIndexes(name):
  diff_array = getDiff(name)
  low=5
  high=len(diff_array)+1
  mid=0
  print('getting Indexes')

  while low<high:
    print(low,high)
    mid=(low+high+1)>>1
    sm_diff_array = smooth(diff_array, mid)
    frame_indexes = np.asarray(argrelextrema(sm_diff_array, np.greater))[0]

    if len(frame_indexes)>=10:
      low=mid
    else:
      high=mid-1


  print(low)
  sm_diff_array = smooth(diff_array, low)
  frame_indexes = np.asarray(argrelextrema(sm_diff_array, np.greater))[0]
  print(len(frame_indexes))

  return frame_indexes

In [ ]:
def worker(low,high,ind):
  size=high-low
  fp=open('extracted'+str(ind)+'.txt','a')

  print(ind,low,high)
  for i in range(low,high):
    videoPath=vids[i]
    data_output_dir=vids[i]+'_key_frames'

    #print('A')
    cap = cv2.VideoCapture(videoPath)

    if not cap.isOpened(): 
      print (ind,"could not open :",videoPath)
      failed.append(vids[i])
      continue
    print(ind,'B')
    frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(ind,'frames '+str(frames))

    
    try:
      
      if os.path.isdir(data_output_dir):
        print('len  ',len(os.listdir(data_output_dir)) , min(frames,frameToExtract))
        if len(os.listdir(data_output_dir)) == min(frames,frameToExtract):
          extracted.append(vids[i])
          fp.write(vids[i]+'\n')
          continue
      else:
        os.mkdir(data_output_dir)
      
      print(ind,'D')

      if frames>frameToExtract:
        frame_indexes = getIndexes(videoPath)

        cap = cv2.VideoCapture(str(videoPath))
        ret, curr_frame = cap.read()

        i2 = 1
        j2 = 0

        while(ret and j2<len(frame_indexes)):
          if i2 == frame_indexes[j2]:
            cv2.imwrite(os.path.join(data_output_dir,str(j2)+'.png'),curr_frame)
            j2 = j2 + 1

          i2 = i2 + 1
          ret, curr_frame = cap.read()
        
        cap.release()
      
      else:
        success,image = cap.read()
        counter = 0
        while success:
          cv2.imwrite(os.path.join(data_output_dir,str(counter))+'.png', image)
          counter+=1     
          success,image = cap.read()

      #print('E')
      cap.release()
      extracted.append(vids[i])
      fp.write(vids[i]+'\n')
      #print('F')


    except Exception as e:
      print(ind,str(e))
      print (ind,"could not extract key frame from :"+videoPath + " with "+str(cap.get(cv2.CAP_PROP_FRAME_COUNT))+" frames")
      failed.append(vids[i])

    print(ind,(i+1-low)/size*100)

  fp.close()

In [ ]:
failed=[]
extracted=[]
worker(0,len(vids),0)

In [ ]:
print(len(failed),len(extracted))

In [ ]:
if len(failed):
    print('Now i have no solution')
else:
    print('Done')